<a href="https://colab.research.google.com/github/beliabaez/CE888_2020/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
df = pd.read_csv('jester-data-1.csv')


In [7]:
arr = df.values
print(arr.shape)

(24982, 101)


In [8]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1835282 (1835282,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])


In [0]:
new_arr, idx = replace(arr, 0.1)

In [12]:
rated = np.where(arr!=99)
new_rated = np.where(new_arr!=99)
print(len(rated[0]), rated[1].shape)


1835282 (1835282,)


In [13]:
n_latent_factors = 2

user_ratings = arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

print(user_ratings)

[[100.     4.08  -0.29 ...   0.34  -4.32   1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 [ 48.    99.     8.35 ...  99.    99.    99.  ]
 ...
 [ 39.    99.    99.   ...  99.    99.    99.  ]
 [ 37.    99.    99.   ...  99.    99.    99.  ]
 [ 72.     2.43   2.67 ...  99.    99.    99.  ]]


In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]

    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values

    
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [15]:
sgd()

58.192510387441274
24.642861436394345
24.305325319839763
18.889642317127663
18.040546638508747


In [16]:
predictions = latent_user_preferences.dot(latent_item_features.T)

values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(100.0, 94.50439030524315)","(4.08, 2.8003613331004726)","(-0.29, 2.3334792940196074)","(6.36, 2.384693892037825)","(4.37, 0.7051341257989209)","(-2.38, 1.896908665158533)","(-9.66, 3.8727365455769243)","(-0.73, 0.8455693766317764)","(-5.34, 0.12944284388403643)","(8.88, 1.4918201703366827)","(9.22, 3.152061168939095)","(6.75, 4.041126343957512)","(8.64, 3.9153093676644324)","(4.42, -1.1781340154045372)","(7.43, 3.852896058448883)","(4.56, -1.3531379029039348)","(-0.97, -2.880388688768693)","(4.66, -0.26972545626173644)","(-0.68, 0.3312772200739288)","(3.3, 1.6646126842045736)","(-1.21, 0.39976913690197685)","(0.87, 4.606250108929893)","(8.64, 3.1801496059244934)","(8.35, 2.246964664918998)","(9.17, 0.26174336356258066)","(0.05, 2.651706444609915)","(7.57, 3.675805374522572)","(4.71, 5.428115365898386)","(0.87, 3.7477770049374897)","(-0.39, 5.135725076126765)","(6.99, 1.7946320711603971)","(6.5, 4.564155771772723)","(-0.92, 5.395688455594636)","(7.14, 0.6951346979078955)","(9.03, 3.201051749040921)","(-1.8, 5.328732654416944)","(0.73, 5.439403942928452)","(7.09, 0.33450243266053037)","(3.4, 3.3940734066866716)","(-0.87, 3.4674952178274925)",...,"(-6.7, 4.88307654225307)","(-3.35, 5.315933695915055)","(-9.03, 2.2787238570514603)","(4.47, 1.209555075464661)","(4.08, 4.6487525838790855)","(-3.83, 4.81784584288883)","(8.74, 1.1386827439453469)","(1.12, 4.854466333501306)","(0.78, 4.947359903566391)","(7.52, 2.685700104861624)","(-5.0, 0.986232348558292)","(2.77, 4.49582017094355)","(8.3, 3.083153849398122)","(7.77, 0.4290482990464203)","(7.33, 1.5417705712484302)","(6.21, 4.136577038902922)","(7.72, 2.6537059295799716)","(8.98, 3.6963928781438486)","(8.64, 2.20912257588834)","(8.2, 3.2586809645203036)","(3.93, 3.6721182964603485)","(4.85, 2.81653536354043)","(4.85, 3.8923282488201534)","(6.07, 2.6371625532686456)","(8.98, 2.928272368947447)","(4.51, 1.9627749343322518)","(-0.05, 3.552513245718048)","(3.69, 3.7655348100289907)","(4.56, 4.993666016152154)","(0.58, 2.6104390663886807)","(2.82, 3.66685040785451)","(-4.95, 3.02993614271227)","(-0.29, 4.189645678500185)","(7.86, 2.883034964757796)","(-0.19, 2.7225183483463877)","(-2.14, 3.2617042529000995)","(3.06, 3.3119612879201794)","(0.34, 2.7411576798916135)","(-4.32, 1.8470053534730455)","(1.07, 3.3261500932352335)"
1,"(49.0, 48.97187901010545)","(99.0, 6.477927479878189)","(99.0, 6.968922544308726)","(99.0, 7.090430339003999)","(99.0, 7.006168462322272)","(9.03, 5.253507577534295)","(9.27, 8.438247479090249)","(9.03, 4.437470613707014)","(9.27, 2.5006787666540475)","(99.0, 7.178025129228249)","(99.0, 6.769706076789631)","(7.33, 8.134783570770992)","(7.57, 8.78501800685963)","(9.37, 2.347555731515182)","(6.17, 8.787546952410803)","(-6.36, 1.534875601684235)","(-6.89, 1.034073272365779)","(-7.86, 3.0372736792515047)","(9.03, 3.621802482933569)","(9.03, 5.417525069465733)","(9.03, 5.132314464712675)","(7.28, 8.570138210221177)","(99.0, 8.172051114605056)","(8.25, 7.335319258844149)","(99.0, 6.277207234089855)","(99.0, 7.91551452432037)","(7.48, 8.55355645894446)","(7.28, 7.742640652221792)","(7.28, 8.081871416215424)","(8.93, 7.337616168044936)","(99.0, 7.978243087697579)","(6.17, 8.185969315909547)","(7.28, 7.735197470345056)","(99.0, 6.9054869461802095)","(99.0, 8.028951051935978)","(8.98, 8.008248291909945)","(7.33, 7.338630280105105)","(99.0, 5.679087578247471)","(6.17, 7.695658951111834)","(9.08, 8.450094445521227)",...,"(6.46, 8.600210828412534)","(7.28, 8.228657102734147)","(99.0, 7.556201736596744)","(99.0, 6.478069801456117)","(7.04, 8.197746064304004)","(7.28, 8.089175144935634)","(99.0, 6.7714541303426214)","(7.28, 7.878790951830885)","(8.25, 8.272434891493495)","(99.0, 7.7923572967801435)","(99.0, 6.0232583192727915)","(99.0, 7.77306843